In [ ]:
from datascience import *
%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np
import warnings
warnings.simplefilter(action='ignore')

# Lecture 11

## Bike Sharing 

In [ ]:
trip = Table.read_table('trip.csv')
trip.show(3)

Trip Data: 

* Trip ID - id for each trip
* Duration - duration of trip (seconds)
* Start Date
* Start Station
* Start Terminal - another identifier for the start station
* End Date
* End Station - another identifier for end station
* Bike #
* Subscriber type
* Zip code of user

## Distribution of Durations

One thing we can look at is the distribution of the trip duration.

In [ ]:
trip.hist('Duration')

<br><br>
**Exercise**:  Focus on trips under 30 minutes (1800 seconds).   Create a histogram of the duration of the commutes.

In [ ]:
commute = ...


<br><details><summary>Click for Solution</summary> <br>
    
```python
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')
```
    
<br></details><br>

<br><br><br>
**Exercise:** Create histogram with bins ranging from 0 to 1800 in increments of 250

<br><details><summary>Click for Solution</summary> <br>
    
```python
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')
```
    
<br></details><br>

<br><br>

**Exercise:** What percentage of the data (trips) have a duration between 250 and 500 seconds. 

In [ ]:
# Approx percent of people/trips who have 
# a ride duration between 250 and 500 seconds
# "between" = [250, 500) 



In [ ]:
# Exact answer: 

commute.where('', ).<>

In [ ]:
commute.num_rows

# <br><br><br>
**Exercise:** Create histogram with more fine grained bins

In [ ]:
commute.hist('Duration', bins=60, unit='Second')

<br><br>

## Start and End Stations

<br><br>

**Exercise** Determine which start station is most common. 

In [ ]:
# Most common start station
starts = ...
starts

<br><details><summary>Click for Solution</summary> <br>
    
```python
starts = commute.group('Start Station').sort('count', descending=True)
starts
```
    
<br></details><br>

<br>

**Exercise:**  Determine the number of trips between stations. 

In [ ]:
# Numbers of trips between stations


<br><details><summary>Click for Solution</summary> <br>

```python
commute.group(['Start Station', 'End Station'])
```

or
    
```python
commute.pivot('Start Station', 'End Station')
```
    
<br></details><br>

<br> 

**Exercise:** Determine the average duration of the trips between all stations. 

In [ ]:
# Average durations of trips between stations



<br><details><summary>Click for Solution</summary> <br>
    
```python
commute.pivot('Start Station', 'End Station', values='Duration', collect=np.average)
```
    
<br></details><br>

<br><br>

## Fastest Trips between Stations 

<br><br>

**Exercise** How can we find the fastest trip ever between each pair of stations?

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
# How would we calculate the best case scenario time between stations?

shortest = ...
shortest.show(5)

<br><details><summary>Click for Solution</summary> <br>
    
```python
shortest = duration.group(['Start Station', 'End Station'], min)
shortest.show(5)
```
    
<br></details><br>

<br><br>

## Discussion question

<br>

**Exercise** Find the 5 stations closest to Civic Center BART by minimum trip time.

In [ ]:
'Civic Center BART'

<br><details><summary>Click for Solution</summary> <br>
    
```python
from_cc = (
    shortest
    .where('Start Station', are.containing('Civic Center BART'))
    .sort('Duration min')
)
from_cc.take(np.arange(5))
```
    
<br></details><br>

<br>

## Maps

In [ ]:
# Geographical data on the stations
stations = Table.read_table('station.csv').drop(4, 6)
stations

In [ ]:
# Get data in a format needed for create a map
# Focused on stations in San Francisco

sf_stations = stations.where('landmark', are.equal_to('San Francisco'))
sf_stations_map_data = (sf_stations
 .select('lat', 'long', 'name')
 .relabeled('name', 'labels')
                       )
sf_stations_map_data.show(3)

In [ ]:
Marker.map_table(sf_stations_map_data)

In [ ]:
Circle.map_table(sf_stations_map_data)

<br><br>

### Discussion question

We now have a number of different tables, e.g.,  

* `from_cc` has start (Civic Center BART) and end station with the minimum duration.
* `sf_stations_map_data` has lat, long, and labels of the station names.

<br> 

**Exercise** Map all stations within 4 minutes (minimum ride time) of Civic Center.

In [ ]:
from_cc.show(3)

In [ ]:
sf_stations_map_data.show(3)

1. Filter out from `from_cc` all the stations that are with in 4 minutes ride from the Civic center
2. Join/Pivot ??
3. Markers.map_table

In [ ]:
close_cc = from_cc.<>
close_markers = (sf_stations_map_data
      <>
Marker.map_table(close_markers)

<br><details><summary>Click for Solution</summary> <br>
    
```python
close_cc = from_cc.where('Duration min', are.below(4 * 60))
close_markers = (sf_stations_map_data
      .join('labels', close_cc, 'End Station')
      .select('lat', 'long', 'labels'))
Marker.map_table(close_markers)
```
    
<br></details><br>

<br><br>
## Add Colors!

Choose marker colors by the minimum time from Civic Center

In [ ]:
minutes = np.round(from_cc.column("Duration min") / 60)
print(min(minutes), max(minutes))

In [ ]:
colors = Table().with_columns(
    "minutes", np.arange(15),
    "colors",  ["orange", "orange", 
                "lightgreen", "green", "darkgreen", 
                "lightgray", "lightgray", "lightgray", 
                "gray", "gray", "gray", "darkgray",
                "black", "black", "black"])

colors_cc = (from_cc
 .with_column("Minutes", minutes)
 .join("Minutes", colors, "minutes"))

colored_markers = (sf_stations_map_data
      .join('labels', colors_cc, 'End Station')
      .select('lat', 'long', 'labels', 'colors'))
Marker.map_table(colored_markers)

## Example Midterm Questions

For each of the following, write a Python expression that returns the given data:

1. The name of the station where most rentals ended

2. The number of stations for which the average duration of trips ending at that station was more than 10 minutes

3. The number of stations that have more than 500 starts and more than 500 ends.